In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/states_2023/raw/ACSDT5Y2023.B18102-Data.csv', index_col=False)

In [3]:
# remove the last unnamed NA column
df = df.loc[:, ~df.columns.str.match("^Unnamed")]

In [4]:
df.head()

,Geography,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!Male:,Margin of Error!!Total:!!Male:,Estimate!!Total:!!Male:!!Under 5 years:,Margin of Error!!Total:!!Male:!!Under 5 years:,Estimate!!Total:!!Male:!!Under 5 years:!!With a hearing difficulty,Margin of Error!!Total:!!Male:!!Under 5 years:!!With a hearing difficulty,...,Estimate!!Total:!!Female:!!65 to 74 years:!!With a hearing difficulty,Margin of Error!!Total:!!Female:!!65 to 74 years:!!With a hearing difficulty,Estimate!!Total:!!Female:!!65 to 74 years:!!No hearing difficulty,Margin of Error!!Total:!!Female:!!65 to 74 years:!!No hearing difficulty,Estimate!!Total:!!Female:!!75 years and over:,Margin of Error!!Total:!!Female:!!75 years and over:,Estimate!!Total:!!Female:!!75 years and over:!!With a hearing difficulty,Margin of Error!!Total:!!Female:!!75 years and over:!!With a hearing difficulty,Estimate!!Total:!!Female:!!75 years and over:!!No hearing difficulty,Margin of Error!!Total:!!Female:!!75 years and over:!!No hearing difficulty
0,0400000US01,Alabama,4969866,971,2394520,1670,150157,935,418,170,...,18183,975,263892,1066,197458,840,34485,1160,162973,1337
1,0400000US02,Alaska,704133,1934,360244,1851,24436,245,120,59,...,2425,309,29301,411,16355,231,3490,323,12865,387
2,0400000US04,Arizona,7154961,1212,3538403,1611,203918,257,1095,313,...,23941,1195,386459,1296,303345,620,53490,1655,249855,1717
3,0400000US05,Arkansas,2977706,755,1457529,1468,93143,613,598,213,...,11687,902,151061,1020,117069,628,22323,1004,94746,1078
4,0400000US06,California,38761738,3244,19237525,3563,1132401,699,6750,827,...,91461,2193,1763772,2343,1376515,1257,249357,3823,1127158,3853


In [5]:
df.columns

Index(['Geography', 'Geographic Area Name', 'Estimate!!Total:',
       'Margin of Error!!Total:', 'Estimate!!Total:!!Male:',
       'Margin of Error!!Total:!!Male:',
       'Estimate!!Total:!!Male:!!Under 5 years:',
       'Margin of Error!!Total:!!Male:!!Under 5 years:',
       'Estimate!!Total:!!Male:!!Under 5 years:!!With a hearing difficulty',
       'Margin of Error!!Total:!!Male:!!Under 5 years:!!With a hearing difficulty',
       'Estimate!!Total:!!Male:!!Under 5 years:!!No hearing difficulty',
       'Margin of Error!!Total:!!Male:!!Under 5 years:!!No hearing difficulty',
       'Estimate!!Total:!!Male:!!5 to 17 years:',
       'Margin of Error!!Total:!!Male:!!5 to 17 years:',
       'Estimate!!Total:!!Male:!!5 to 17 years:!!With a hearing difficulty',
       'Margin of Error!!Total:!!Male:!!5 to 17 years:!!With a hearing difficulty',
       'Estimate!!Total:!!Male:!!5 to 17 years:!!No hearing difficulty',
       'Margin of Error!!Total:!!Male:!!5 to 17 years:!!No hearing diffi

In [6]:
def clean_col(name):
    if name.startswith("Estimate!!"):
        name = name.replace("Estimate!!", "")
    elif name.startswith("Margin of Error!!"):
        name = name.replace("Margin of Error!!", "moe_")
    
    name = name.replace("Estimate!!", "").replace("Margin of Error!!", "")
    name = name.replace("Estimate!!Total:", "total")
    name = name.replace("Male:", "male")
    name = name.replace("Female:", "female")
    name = name.replace("Under 5 years:", "under5")
    name = name.replace("5 to 17 years:", "5to17")
    name = name.replace("18 to 34 years:", "18to34")
    name = name.replace("35 to 64 years:", "35to64")
    name = name.replace("65 to 74 years:", "65to74")
    name = name.replace("75 years and over:", "75plus")
    name = name.replace("With a hearing difficulty", "hearing_diff")
    name = name.replace("No hearing difficulty", "no_hearing_diff")

    name = name.replace("!!", "_").replace(":", "").replace(" ", "_").lower()
    
    return name

df.columns = [clean_col(c) for c in df.columns]

In [7]:
# adding state abbreviations
state_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "Puerto Rico": "PR"
}

df["state_short"] = df["geographic_area_name"].map(state_abbrev)

cols = list(df.columns)
i = cols.index("geographic_area_name")
cols.insert(i + 1, cols.pop(cols.index("state_short")))
df = df[cols]

In [8]:
df.head()

,geography,geographic_area_name,state_short,total,moe_total,total_male,moe_total_male,total_male_under5,moe_total_male_under5,total_male_under5_hearing_diff,...,total_female_65to74_hearing_diff,moe_total_female_65to74_hearing_diff,total_female_65to74_no_hearing_diff,moe_total_female_65to74_no_hearing_diff,total_female_75plus,moe_total_female_75plus,total_female_75plus_hearing_diff,moe_total_female_75plus_hearing_diff,total_female_75plus_no_hearing_diff,moe_total_female_75plus_no_hearing_diff
0,0400000US01,Alabama,AL,4969866,971,2394520,1670,150157,935,418,...,18183,975,263892,1066,197458,840,34485,1160,162973,1337
1,0400000US02,Alaska,AK,704133,1934,360244,1851,24436,245,120,...,2425,309,29301,411,16355,231,3490,323,12865,387
2,0400000US04,Arizona,AZ,7154961,1212,3538403,1611,203918,257,1095,...,23941,1195,386459,1296,303345,620,53490,1655,249855,1717
3,0400000US05,Arkansas,AR,2977706,755,1457529,1468,93143,613,598,...,11687,902,151061,1020,117069,628,22323,1004,94746,1078
4,0400000US06,California,CA,38761738,3244,19237525,3563,1132401,699,6750,...,91461,2193,1763772,2343,1376515,1257,249357,3823,1127158,3853


In [9]:
# export df to a clean csv
df.to_csv("../data/states_2023/clean/acs_states2023.csv", index=False, na_rep="")